# Assignment 1

This assignment will involve the creation of a spellchecking system and an evaluation of its performance. You may use the code snippets provided in Python for completing this or you may use the programming language or environment of your choice

Please start by downloading the corpus `holbrook.txt` from Blackboard

The file consists of lines of text, with one sentence per line. Errors in the line are marked with a `|` as follows

    My siter|sister go|goes to Tonbury .
    
In this case the word 'siter' was corrected to 'sister' and the word 'go' was corrected to 'goes'.

In some places in the corpus two words maybe corrected to a single word or one word to a multiple words. This is denoted in the data using underscores e.g.,

    My Mum goes out some_times|sometimes .
    
For the purpose of this assignment you do not need to separate these words, but instead you may treat them like a single token.

*Note: you may use any functions from NLTK to complete the assignment. It should not be necessary to use other libraries and so please consult with us if your solution involves any other external library. If you use any function from NLTK in Task 6 please include a brief description of this function and how it contributes to your solution.*

## Task 1 (10 Marks)

Write a parser that can read all the lines of the file `holbrook.txt` and print out for each line the original (misspelled) text, the corrected text and the indexes of any changes. The indexes refers to the index of the words in the sentence. In the example given, there is only an error in the 10th word and so the list of indexes is [9]. It is not necessary to analyze where the error occurs inside the word.

Then split your data into a test set of 100 lines and a training set.

In [16]:
#importing the required libraries
import re
import nltk as nltk
import pandas as pd
from itertools import chain
from nltk import sent_tokenize
from nltk import ngrams, FreqDist
from collections import defaultdict
import string
from nltk.metrics.distance import edit_distance
from nltk.tokenize import RegexpTokenizer
#from nltk.tokenize.moses import MosesDetokenizer

In [17]:
#Intialising lists to store tokens and the data that is imported from the holbrook.txt file
word_token = []
data = []

#Reading the file line by line
with open('holbrook.txt') as f:
    data = f.readlines()

#tokenizing the read data storing them as sentences
for i in range(0,len(data)):
    #print(data[i])
    sent_token=nltk.sent_tokenize(data[i])
    #tokenizing the sentencesz into words
    for q in sent_token:
        word_token.append(nltk.word_tokenize(q))
        
#Creating list to store the final processed data
finaldata = []
finaldata1= []

#Segregating the original and the corrected versions along with indexes
for r in word_token:
    #Initialising lists. ori for original text i.e text with wrong spellings. cor text with correct spelling and index.
    ori=[]
    cor=[]
    ind=[]
    #Splitting on the basis of |.
    for t in r:
        if '|' in t:
            #finding the index of the token containing |
            ind.append([r.index(t)])
            
        split_token=t.split("|")
        
        #Storing the left part of the | into ori - list of sentences with wrong spellings
        #Storing the right part of the | into cor - list of sentences with correct spellings
        if len(split_token) == 1:
            ori.append(split_token)
            cor.append(split_token)
            
        else:
            #Storing all others tokens without | into both ori and cor lists.
            ori.append([split_token[0]])
            cor.append([split_token[1]])

#Initialising an empty dictionary
    dicts={}
#Storing the respective sentences under their keys
    dicts['original'] = list(chain(*ori))
    dicts['corrected'] = list(chain(*cor))
    dicts['indexes'] = list(chain(*ind))
    
#appending the dictionary withe values into a list
    finaldata.append(dicts)

#Checking the final result.
#print(finaldata[2])
#print(type(finaldata))


In [18]:
#Verifying Answer
assert(finaldata[2] == {'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 
   'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 
    'indexes': [9]
})

In [4]:
#Splitting the data into testing and training sets
# test_a = finaldata1[:100]
# train_a = finaldata1[100:]
# print(test_a)

In [5]:
#Splitting the data into testing and training sets
test = finaldata[:100]
train = finaldata[100:]
#print(test)

In [19]:
#Splitting the data into testing and training sets
test_1= finaldata[:100]
train_1 = finaldata[100:]
#print(test_1)

The counts and assertions given in the following sections are based on splitting the training and test set as follows

## **Task 2** (10 Marks): 
Calculate the frequency (number of occurrences), *ignoring case*, of all words and bigrams (sequences of two words) from the corrected *training* sentences:

In [7]:
#Extracting only the sentences under 'corrected' key from the training set
correctedset=[]

for i in range(0,len(train)):
     correctedset.append(train[i]['corrected'])

#Calculating Unigram frequency
def unigram(d):
    #Initialising counter variable
    count=0
    #Counting the occurence of token in the set
#     totalunigram=len(correctedset)
    #print(totalunigram)
    for i in correctedset:
        #print(i)
        for j in i:
            #print(j)
            if(d == j):
                count+=1 
    #Returning the frequency a given token
    return(count)

#Calculating bigram frequency
def bigram(bg_term):
    bg_term=str(bg_term)
    #Splitting the term given by the user on space: converting bigrams into two unigrams
    bg_term = bg_term.split(" ")
    #Initialising a counter variable
    totalbigram=0
    count1=0
    #Calculating frequency of bigrams
    for i in correctedset:
        for j in range(0,len(i)):
            #Storing the sentences from the training set in the form of bigrams
            bigramtokens=(i[j-1],i[j])

            #Checking presence of bigram terms in the text and counting each occurence. all the tokens are converted into lowercase
            #inorder to ignore case sensitivity.
            if(bg_term[0].lower() == bigramtokens[0].lower() and bg_term[1].lower() == bigramtokens[1].lower()):
                count1=count1+1
    #Returning the count i.e. frequency
    #print(totalbigram)
    return count1

#Checking results

assert(unigram("me")==87)
assert(bigram("my mother")==17)
assert(bigram("you did")==1)

## **Task 3** (15 Marks): 
[Edit distance](https://en.wikipedia.org/wiki/Edit_distance) is a method that calculates how similar two strings are to one another by counting the minimum number of operations required to transform one string into the other. There is a built-in implementation in NLTK that works as follows:


In [ ]:
from nltk.metrics.distance import edit_distance

# Edit distance returns the number of changes to transform one word to another
print(edit_distance("hello", "hi"))

Write a function that calculates all words with *minimal* edit distance to the misspelled word. You should do this as follows

1. Collect the set of all unique tokens in `train`
2. Find the minimal edit distance, that is the lowest value for the function `edit_distance` between `token` and a word in `train`
3. Output all unique words in `train` that have this same (minimal) `edit_distance` value

*Do not implement edit distance, use the built-in NLTK function `edit_distance`*

In [8]:
#Initialising lists to extract and store the corrected spelling text and the mis spelled text from the dictionary created above.
correctspelledset=[]
mispelledset=[]

#To store each tokens from the sets above
mstokens=[]
cstokens=[]

#To extract and store the unique tokens
unique_train=[]

for i in range(0,len(train)):
    
    #Extracting the text with wrong spelling and storing unique values using 'set'
    mispelledset.append(train[i]['original'])
    mstokens=set(list(chain(*mispelledset)))
    
    #Extracting the text with correct spellings and storing the unique values using 'set'
    correctspelledset.append(train[i]['corrected'])
    cstokens=set(list(chain(*correctspelledset)))

#Storing the unique tokens
unique_train=list(cstokens.union(mstokens))



In [9]:

def get_candidates(token):
    #Initialising a list to store the edit distances
    ed=[]

#Initialising a list to store the tokens with minimum edit distances
    min_ed_tk=[]

#Storing the tokens with which edit distance is calculated
    ed_token=[]
    for a in range(0, len(unique_train)):
        
        #Appending the calculated edit distances to the list ed.
        ed.append(edit_distance(token,unique_train[a]))
        
        #Appending the tokens to the lists ed_token
        ed_token.append(unique_train[a])
        
    #Finding out the minimum edit distance between the tokens
    edmin=min(ed) 
    
   #Extracting ans storing only those tokens with the lowest edit distances.
    for b in range(0,len(ed)):
        
        #Finding the tokens with min edit distance value
        if(ed[b]==edmin):
            min_ed_tk.append(ed_token[b])
            #print(min_ed_tk)
            
    #Returning those tokens which were extracted from the previous step.
    return min_ed_tk
        

#Test your code as follows
assert get_candidates("minde") == ['mine', 'mind']
#assert get_candidates("minde") == ['mind', 'mine']

# Note: For the above assert() sometime for ("minde") get_candidates() return ['mind','mine'] instead of ['mine','mind']

## Task 4 (15 Marks):

Write a function that takes a (misspelled) sentence and returns the corrected version of that sentence. The system should scan the sentence for words that are not in the dictionary (set of unique words in the training set) and for each word that is not in the dictionary choose a word in the dictionary that has minimal edit distance and has the highest *bigram probability*. That is the candidate should be selected using the previous and following word in a bigram language model. Thus, if the $i$th word in a sentence is misspelled we should use the following to rank candidates:

$$p(w_{i+1}|w_i) p(w_i|w_{i-1})$$

For the first and last word of the sentence use only the conditional probabilities that exist.

*Your solution to this should involve `get_candidates`*


# Note: for the sets of candidates with 0 probability for all the individuals in the list i am choosing the candidate at index 0 as the replacement

In [10]:
#Extracting the sentences with correct spelling and storing it seperately
correctdictionary = list(chain(*correctspelledset))

#defining the function to correct the spelling mistakes.
def correct(sentence):
    
    #initialising the required variable
    #this contains the candidate used to used the misspelled word
    a=[]
    
    #this list conatins all the incorrent terms
    incorrect_terms=[]
    
    #Contains the word at the prev index
    w_index_prev=''
    
    #contains word at the next index
    w_index_next=''
    
    #to store Probability
    probability=[]
    
    #to store the candidates returned by the get_candidates() function
    rep_candi=[]
    
    #to store the index of the wrong terms in the sentence
    w_index=0
    
    #these two variables are for backing up the original sentence passed on as an arguement.
    s1=sentence
    s2=sentence
    
##################################################################################################################    
######## Defining a function to find the probability of wrong term when its appearing in between the first and the
######## last word################################################################################################
    
    def prob_middle(rep_candi):
        
        #checking if number of candidates returned by the get_candidates() is more than one.
        if (len(rep_candi)>1):
            #If more than one candidates are returned then iterating over each.
            for j in rep_candi:
                
                #generating bigrams with previous terms
                bi_token_prev=w_index_prev+" "+j
                
                #counting the number of bigrams
                bi_count_prev=bigram(bi_token_prev)
                
                #generating bigrams with the next term
                bi_token_next=j+" "+w_index_next
                
                #counting the number of bigrams
                bi_count_next=bigram(bi_token_next)
                
                #calculating the unigram count
                unicount_next=unigram(j)
                
                #checking if unigram count or bigram count is zero. In such cases putting probability as 0
                if (unicount_next == 0 or bi_count_next==0 or uni_prev==0 or bi_count_prev==0):
                    probability.append(0)
                else:
                    #Else calculating the probability as per the given formula.
                    probability.append((bi_count_prev/uni_prev)*(bi_count_next/unicount_next))

                #Iterating over the list of probabilities and finding out the maximum
                for q in probability:
                    #Checking that probability is max and not equal to zero
                    if (q == max(probability) and q>0):
                        #replacing the wrong word with the corresponding word of the highest probability
                        a=''.join(j)
                        s1[int(w_index)]=a
                        #print(s1)
                    else:
                        #if probability is zero then replacing the wrong word with the first word returned from get_candidates()
                        s1[int(w_index)]=rep_candi[0]
                    
        #If Length of the list of candidates returned by get_candidates() ####
        #function is one then replacing the wrong word with the candidate         
        else:            
            a=''.join(rep_candi)
            s1[int(w_index)]=a
            
###################################################################################################################    
######## Defining a function to find the probability of wrong term when its appearing as the first word ############
###################################################################################################################   

    def prob_first(rep_candi):
        
        maxprob=0
        #When number of candidates returned>1
        if (len(rep_candi)>1):
            #iterating through the list of candidates returned by the get_candidates()
            for j in rep_candi:
                #print(j)
                
                #Generating bigrams with the misspelled word and the next word
                bi_token_next=j+" "+w_index_next
                
                #Calculating Bigram count
                bi_count_next=bigram(bi_token_next)
    
                #calculating the unigram count of the next word
                unicount1=unigram(j)
    
                #checking if unigram count or bigram count is zero. In such cases putting probability as 0
                if (unicount1 == 0 or bi_count_next==0):
                    probability.append(0)
                else:
                    #Else calculating the probability as per the given formula.
                    probability.append((bi_count_prev/uni_prev)*(bi_count_next/unicount_next))
                    
                #Iterating over the list of probabilities and finding out the maximum    
                for q in probability:
                    #Checking that probability is max and not equal to zero
                    if (q == max(probability) and q>0):
                        #replacing the wrong word with the corresponding word of the highest probability
                        a=''.join(j)
                        s1[int(w_index)]=a
                        #print(s1)
                    else:
                        #if probability is zero then replacing the wrong word with the first word returned from get_candidates()
                        s1[int(w_index)]=rep_candi[0]

        #If Length of the list of candidates returned by get_candidates() ###
        #function is one then replacing the wrong word with the candidate                     
        else:
            a=''.join(rep_candi)
            s1[int(w_index)]=a
        
###################################################################################################################    
######## Defining a function to find the probability of wrong term when its appearing as the last word ############
###################################################################################################################   

    def prob_last(rep_candi):
        max_prob=0
        #When number of candidates returned>1
        if (len(rep_candi)>1):
            
            #iterating through the list of candidates returned by the get_candidates()
            for j in rep_candi:
                #generating bi grams
                bi_token_prev=w_index_prev+" "+j
                
                #Calsulating bigram count
                bi_count_prev=bigram(bi_token_prev)
               
                #calculating the unigram count of the prev. word
                unicount_prev=unigram(bi_token_prev)
            
                #checking if unigram count or bigram count is zero. In such cases putting probability as 0
                if (unicount_prev == 0 or bi_count_prev==0):
                    probability.append(0)
                else:
                    #Else calculating the probability as per the given formula.
                    probability.append((bi_count_prev/uni_prev)*(bi_count_next/unicount_next))
                    
                #Iterating over the list of probabilities and finding out the maximum                  
                for q in probability:
                    if (q == max(probability)and q>0):
                        #replacing the wrong word with the corresponding word of the highest probability
                        a=''.join(j)
                        s1[int(w_index)]=a
                        #print(s1)
                    else:
                        #if probability is zero then replacing the wrong word with the first word returned from get_candidates()
                        s1[int(w_index)]=rep_candi[0]
                               
        #Length of the list of candidates returned by get_candidates() ### 
        #function is one then replacing the wrong word with the candidate    
        else:
            a=''.join(rep_candi)
            s1[int(w_index)]=a
            
############################################################################################################################
############################################################################################################################
    
    #Iterating over the entered sentence
    for i in s1:
        #Checking if a token is correct or not
        if (i in correctdictionary):
            
            #Storing the index
            r_index=s1.index(i)
            
        else:
            #Storing index of the wrong word
            wrong_index=s1.index(i)
            
            #Listing all the incorrect terms
            incorrect_terms.append(i)
            
            #checking if the wrong word is at the first place
            if (s1.index(i)==0):
                #Storing index
                w_index=s1.index(i)
                #Storing candidates returned in a list
                rep_candi=get_candidates(i)
                #calling the function prob_first() defined above
                prob_first(rep_candi)
                
            #Checking if the wrong word is at the last index
            elif(s1.index(i)==len(s1)-1):
                
                #Storing the index
                w_index=s1.index(i)
                
                #Getting the candidates
                rep_candi=get_candidates(i)
                
                #calling prob_last function defined above
                prob_last(rep_candi)
              
            #Checking if wrong word appear in between
            else:
                
                #Storing index
                w_index=s1.index(i)
                
                #Storing the prev word
                w_index_prev=s1[s1.index(i)-1]
                
                #Storing previous word unigram count
                uni_prev=(unigram(w_index_prev))
                
                #Storing the next word
                w_index_next=s1[s1.index(i)+1]
                
                #storint he candidates
                rep_candi=get_candidates(i)
                #Calling the function prob_middle() defined above
                prob_middle(rep_candi)
                
    #Returning the result.
    return s1
s=["this","whitr","cat"]

correct(s)
assert(correct(["this","whitr","cat"]) == ['this','white','cat']) 

## **Task 5** (10 Marks): 
Using the test corpus evaluate the *accuracy* of your method, i.e., how many words from your system's output match the corrected sentence (you should count words that are already spelled correctly and not changed by the system).

In [15]:
#Extracting only the sentences under 'corrected' key from the training set
correctedset_full=[]
for i in range(0,len(finaldata)):
    correctedset_full.append(finaldata[i]['corrected'])

#Extracting the original part
mispelledset_full=[]
for i in range(0,len(finaldata)):
    mispelledset_full.append(finaldata[i]['original'])


correctedset_test=[]
for i in range(0,len(test)):
    correctedset_test.append(test[i]['corrected'])

mispelledset_test=[]
for i in range(0,len(test)):
    mispelledset_test.append(test[i]['original'])



#Defining accuracy function
def accuracy(test1):
    pred_count=0
    
    
    for i in range(0,len(test1)):
     
        s_result=correct(test1[i])
      
        for i1 in range (0,len(s_result)):
            for j in range(0,len(correctedset_test)):
                if(s_result == correctedset_test[j]):
                     #initialising count variable
      
                    pred_count=pred_count+1
                    
    #Calculating accuracy     
    accuracy=(pred_count*100/len(correctedset_test))
     
    return accuracy

accuracy(mispelledset_test)


85.0

## **Task 6 (35 Marks):**

Consider a modification to your algorithm that would improve the accuracy of the algorithm developed in Task 3 and 4

* You may resources beyond those provided on Blackboard.
* You must **not use the test data** in this task.
* Marks will be awarded based on how interesting the proposed improvement is. 
* Please provide a short text describing what you intend to do and why. 
* Full marks for this section may be obtained without an implementation, but an implementation is preferred.


## Note: Before running the modification part please re run the cells from 1 - 6. This is basically because on running the code over and over again the accuracy functions return ambiguous values.

## Possible Modifications:
1. Removing the dots present as a single element and removing the digits: I have tried to implement this and the accuracy improved from 85% to  96%. T
2. Another modification done is instead of calculating the probability I have directly matched the candidates returned with the correct set. if the value matched we ignore, otherwise we the candidate with the token in the corrected set.
3. A better enhancenment would be considering the context and synonyms of the word. This works as:

Modification: for all the words in the corpus define a cluster that contains words having similar meanings and usage -> for a wrong word, find out the cluster with which it matches the most -> then use the get_candidates() to fetch the candidates with minimum edit distance -> then generate bigrams of the wrong word with each of the candidates returned and calculate the probability-> finally replace the word with the candidate with max probability.

In [20]:
#Extracting Correct set
correctedset_1test=[]
for i in range(0,len(test_1)):
    correctedset_1test.append(test_1[i]['corrected'])

#Extracting Original set
mispelledset_1test=[]
for i in range(0,len(test_1)):
    mispelledset_1test.append(test_1[i]['original'])


corr_terms=[]
for c1 in correctedset_1test:

     for c2 in c1:
        if c2 != '.':
            corr_terms.append(c2)


In [28]:
#defining correct_enh function for the modification
def correct_enh(input_test):
    list1=[]
    #print(input_test)
    
   #function to check the similarity ofm candidates with the corrected set and replacing the mispelled word accordingly
    def function1(ind,candidates_for_wrong_terms):
        replacement=[]
        
        #for cases when more than one candidates are returned
        if ( len (candidates_for_wrong_terms) > 1):
            #iterating over the candidates
            for y1 in candidates_for_wrong_terms:
                #iterating over the correct terms
                for y2 in corr_terms:
                    #matching similarity
                    if y1 == y2:
                            replacement.append(y1)
                    else:
                            replacement.append(y2)

        #returning the replacement word.
        return replacement 
    
    wrong_term=[]
    #iterating over the input set
    for q1 in input_test:
        #iterating over the corrected list
        for q2 in q1:
            #storing the wqrong terms
            if (q2 not in correctedset_1test):
                wrong_term.append(q2)

    
    wrong_term1=[]
    for w1 in wrong_term:
        #Removing dots and digits
        if w1 != '.' and w1 !=0 and w1!=1 and w1!=2 and w1!=2 and w1!=4 and w1!=5 and w1!=6 and w1!=7 and w1!=8 and w1!=9:
            wrong_term1.append(w1)
    #storing the index
    index_wrong_term=0
    candidates_for_wrong_terms=[]
    for e1 in wrong_term1:
        if e1 not in corr_terms:
            index_wrong_term=wrong_term1.index(e1)
            candidates_for_wrong_terms=get_candidates(e1)
            #caling the function defined above to find the replacement of the wrong word

            list1.append(function1(index_wrong_term,candidates_for_wrong_terms))
            
            
    #Calculating the accuracy
    acc_m= (len(list1)*100/len(correctedset_1test))
    #returning Accuracy.
    return acc_m

correct_enh(mispelledset_1test)

96.0

## **Task 7 (5 Marks):**

Repeat the evaluation (as in Task 5) of your new algorithm and show that it outperforms the algorithm from Task 3 and 4

In [29]:
#Calculating Accurracy for the enhancement.

def accuracy_1(test1):
    #calling the function in the previous cell
    accuracy=correct_enh(test1)

    return accuracy
#passing the mispelled set
accuracy_1(mispelledset_1test)

96.0